In [27]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')


In [34]:
import pandas as pd
from mlforecast import MLForecast
from xgboost import XGBRegressor
from utilsforecast.plotting import plot_series
from matplotlib import pyplot as plt
from window_ops.rolling import rolling_mean, rolling_std, rolling_min, rolling_max
from mlforecast.target_transforms import Differences
from sklearn.metrics import mean_squared_error, accuracy_score ,root_mean_squared_error
import numpy as np
from scipy import signal
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from prophet import Prophet
from sklearn.base import BaseEstimator, RegressorMixin
from statsmodels.tsa.seasonal import STL
from lightgbm import LGBMRegressor

In [5]:
df=pd.read_csv('MLTempDataset.csv')
df.drop(columns=['Unnamed: 0',],inplace=True)#'Datetime1'
df1=df.copy()
df1['unique_id']='h1'
# df1['unique_id']=
df1['ds']=pd.to_datetime(df['Datetime'])
df1['y']=df['DAYTON_MW']
df1.drop(columns=['Datetime','DAYTON_MW','Datetime1'],inplace=True)
df1.head()

,unique_id,ds,y
0,h1,2022-01-04 00:00:00,20.867
1,h1,2022-01-04 01:00:00,21.000
2,h1,2022-01-04 02:00:00,20.867
3,h1,2022-01-04 03:00:00,20.650
4,h1,2022-01-04 04:00:00,20.400


In [11]:
chunks=[]
r=int(df1.shape[0]/5)
for i in range(4):
    a=df1[i*r:(i+1)*r]
    chunks.append(a)
chunks.append(df1[4*r:])

In [15]:
train=chunks[0]
test=chunks[1]

In [18]:
from scipy.signal import periodogram

frequencies, spectrum = periodogram(df1['y'])
max_idx = np.argmax(spectrum)
second_max_idx = np.argsort(spectrum)[-2]  # Get the second largest index

# Get the maximum and second maximum values
max_value = spectrum[max_idx]
second_max_value = spectrum[second_max_idx]

# Get the corresponding frequencies
max_freq = frequencies[max_idx]
second_max_freq = frequencies[second_max_idx]

1/max_freq, 1/second_max_freq

# plt.figure(figsize=(14, 7))
# plt.plot(frequencies, spectrum)
# plt.title('Periodogram')
# plt.xlabel('Frequency')
# plt.ylabel('Power')
# plt.show()

(np.float64(3338.0), np.float64(667.6))

In [19]:
seasonal_period=int(1/second_max_freq)

In [22]:
stl = STL(train['y'],period=seasonal_period)
result=stl.fit()
# result.plot()
trend=result.trend
trend.head()
seasonal=result.seasonal
a=pd.DataFrame(trend)
a.shape

(1335, 1)

In [23]:
a['ds']=df1['ds']
a['unique_id']=df1['unique_id']
a['y']=a['trend']
a=a.dropna().reset_index(drop=True)
a.head()

,trend,ds,unique_id,y
0,16.450931,2022-01-04 00:00:00,h1,16.450931
1,16.453445,2022-01-04 01:00:00,h1,16.453445
2,16.455960,2022-01-04 02:00:00,h1,16.455960
3,16.458475,2022-01-04 03:00:00,h1,16.458475
4,16.460990,2022-01-04 04:00:00,h1,16.460990


In [24]:
a.shape

(1335, 4)

In [30]:
model=XGBRegressor(random_state=990,learning_rate=0.01,n_estimators=500,max_depth=10,reg_lambda=0.2)
model2=LGBMRegressor(boosting_type='gbdt',num_leaves=31,learning_rate=0.1,n_estimators=100,random_state=990,verbose=-1)
ftsc=MLForecast(
    models=[model,model2],
    freq='H',
    lags=[seasonal_period],
    lag_transforms={
        # 30:[(rolling_mean,30),(rolling_std,30),(rolling_min,30),(rolling_max,30)],
        # 168:[(rolling_mean,168),(rolling_std,168),(rolling_min,168),(rolling_max,168)],
        seasonal_period:[(rolling_mean,seasonal_period),(rolling_std,seasonal_period),(rolling_min,seasonal_period),(rolling_max,seasonal_period)],
    #     350:[(rolling_mean,350),(rolling_std,350),(rolling_min,350),(rolling_max,350)],
    #     720:[(rolling_mean,720),(rolling_std,720),(rolling_min,720),(rolling_max,720)]
    },
    # target_transforms=[Differences([350])],
    date_features=['hour','day','weekday','month','year'],
)
# ftsc.fit(train,id_col='unique_id',time_col='ds',target_col='y')
# min_child_weight=
# ,gamma=
# ,subsample=
# objective="reg:squarederror",

In [32]:
ftsc.fit(train,id_col='unique_id',time_col='ds',target_col='y')

MLForecast(models=[XGBRegressor, LGBMRegressor], freq=H, lag_features=['lag667', 'rolling_mean_lag667_window_size667', 'rolling_std_lag667_window_size667', 'rolling_min_lag667_window_size667', 'rolling_max_lag667_window_size667'], date_features=['hour', 'day', 'weekday', 'month', 'year'], num_threads=1)

In [33]:
predictions = ftsc.predict(test.shape[0])

In [44]:
seasonalx = np.tile(seasonal[-seasonal_period:], int(np.ceil( test.shape[0] / seasonal_period)))[:test.shape[0]]
seasonalx = pd.Series(seasonalx, index=test.index)
seasonalx=pd.DataFrame(seasonalx)
seasonalx['ds']=pd.to_datetime(test['ds'])
seasonalx['z']=seasonalx[0]
seasonalx['unique_id']='h1'
seasonalx.head()
seasonalx=pd.DataFrame(seasonalx)
seasonalx['y']=seasonalx['z']
seasonalx=seasonalx.reset_index()
seasonalx.head()

,index,0,ds,z,unique_id,y
0,1335,-7.484502,2022-02-28 15:00:00,-7.484502,h1,-7.484502
1,1336,2.095954,2022-02-28 16:00:00,2.095954,h1,2.095954
2,1337,2.335409,2022-02-28 17:00:00,2.335409,h1,2.335409
3,1338,2.124864,2022-02-28 18:00:00,2.124864,h1,2.124864
4,1339,1.955319,2022-02-28 19:00:00,1.955319,h1,1.955319


In [46]:
seasonalx.shape,predictions.head()

((1335, 6),
   unique_id                  ds  XGBRegressor  LGBMRegressor
 0        h1 2022-02-28 15:00:00     11.877375        15.2625
 1        h1 2022-02-28 16:00:00     18.647635        15.2625
 2        h1 2022-02-28 17:00:00     18.647635        15.2625
 3        h1 2022-02-28 18:00:00     18.647635        15.2625
 4        h1 2022-02-28 19:00:00     18.647635        15.2625)

In [49]:
predictions['seas+xgb']=predictions['XGBRegressor']*(seasonalx['y'])
predictions['seas+xgb']=(predictions['seas+xgb']/a['y'].mean())+a['y'].mean()
predictions['seas+lgb']=predictions['LGBMRegressor']*(seasonalx['y'])
predictions['seas+lgb']=(predictions['seas+lgb']/a['y'].mean())+a['y'].mean()
p=predictions.copy()
p.drop(columns=['XGBRegressor','LGBMRegressor'],inplace=True)
p.head()

,unique_id,ds,seas+xgb,seas+lgb
0,h1,2022-02-28 15:00:00,13.238803,11.842073
1,h1,2022-02-28 16:00:00,20.294168,19.903028
2,h1,2022-02-28 17:00:00,20.540331,20.104505
3,h1,2022-02-28 18:00:00,20.323889,19.927353
4,h1,2022-02-28 19:00:00,20.149594,19.784699


In [48]:
root_mean_squared_error(test['y'],predictions['seas+xgb']),root_mean_squared_error(test['y'],predictions['seas+lgb'])

(np.float64(9.454873875768662), np.float64(9.845601992982527))

In [55]:
p['y']=p['seas+lgb']